Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 '10X_PBMC_select_2100',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'worm_neuron_cell_select_2100',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'mouse_ES_cell_select_2100',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 'mouse_bladder_cell_select_2100',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.DataFrame()
dropout = 0.9
lr = 1e-5
layers = [50]
temperature = 0.01
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 1500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)

    for run in range(3):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        dresults = train.run(X,
                             cluster_number,
                             dataset,
                             Y=Y,
                             nb_epochs=20,
                             lr=lr,
                             temperature=temperature,
                             dropout=dropout,
                             evaluate=False,
                             n_ensemble=1,
                             layers=layers,
                             save_to=f"{path}output/real_data/{run}/",
                             save_pred = True)
        dresults["temperature"] = temperature
        dresults["dropout"] = dropout
        dresults["lr"] = lr
        dresults["layers"] = str(layers)
        dresults["run"] = run
#         print(f".", end = "")
        print(f"# {temperature}, {dropout}, {lr}, {layers}", 
              dresults.get('COMBINED_kmeans_ari', ""),
              dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
              dresults.get('leiden_ari_0', ""))
        df = df.append(dresults, ignore_index=True)

        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_baseline.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 1500 genes
# 0.01, 0.9, 1e-05, [50]   0.8617729831887768 0.2576725128954384
# 0.01, 0.9, 1e-05, [50]   0.8730928940264807 0.25673675140001273
# 0.01, 0.9, 1e-05, [50]   0.8695675755745149 0.2208552725398512
>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1500 genes
# 0.01, 0.9, 1e-05, [50]   0.890040682316825 0.9764674490844288
# 0.01, 0.9, 1e-05, [50]   0.9719628280300546 0.9657949249363579
# 0.01, 0.9, 1e-05, [50]   0.8526978032137039 0.9681768120267512
>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 1500 genes
# 0.01, 0.9, 1e-05, [50]   0.8873797744495279 0.20026612850945733
# 0.01, 0.9, 1e-05, [50]   0.8729662687930737 0.20934130662330783
# 0.01, 0.9, 1e-05, [50]   0.8819959284555426 0.17621279471079712
>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 1500 genes
# 0.01, 

In [5]:
df.mean()

dropout         0.900000
kmeans_ari_0    0.689426
kmeans_nmi_0    0.746098
leiden_ari_0    0.526944
leiden_nmi_0    0.690416
lr              0.000010
run             1.000000
temperature     0.010000
dtype: float64

In [4]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_baseline.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"])

,layers,[50]
,lr,0.00001
temperature,dropout,
0.01,0.9,0.689426


In [5]:
df

,dataset,dropout,features_0,kmeans_ari_0,kmeans_nmi_0,layers,leiden_ari_0,leiden_nmi_0,lr,run,temperature
0,Quake_Smart-seq2_Trachea,0.9,"[[-0.18958192, 0.13625321, -0.023023194, 0.252...",0.861773,0.823362,[50],0.257673,0.578862,0.00001,0.0,0.01
1,Quake_Smart-seq2_Trachea,0.9,"[[0.32988557, 0.14088428, -0.030603934, 0.2360...",0.873093,0.841892,[50],0.256737,0.576320,0.00001,1.0,0.01
2,Quake_Smart-seq2_Trachea,0.9,"[[-0.11397525, 0.15452868, -0.07637162, -0.215...",0.869568,0.826321,[50],0.220855,0.556506,0.00001,2.0,0.01
3,Quake_Smart-seq2_Diaphragm,0.9,"[[-0.044101067, 0.17382416, -0.031794988, 0.06...",0.890041,0.863186,[50],0.976467,0.946714,0.00001,0.0,0.01
4,Quake_Smart-seq2_Diaphragm,0.9,"[[0.19135517, 0.009612708, -0.03241578, 0.2574...",0.971963,0.947482,[50],0.965795,0.930536,0.00001,1.0,0.01
5,Quake_Smart-seq2_Diaphragm,0.9,"[[-0.39526477, 0.1512585, -0.07455445, 0.06659...",0.852698,0.863127,[50],0.968177,0.933626,0.00001,2.0,0.01
6,Quake_10x_Spleen,0.9,"[[0.11056912, -0.21663713, 0.08372309, 0.31577...",0.887380,0.783984,[50],0.200266,0.485196,0.00001,0.0,0.01
7,Quake_10x_Spleen,0.9,"[[0.063684724, -0.074820936, -0.10307306, 0.10...",0.872966,0.768931,[50],0.209341,0.487998,0.00001,1.0,0.01
8,Quake_10x_Spleen,0.9,"[[-0.14334916, -0.20600854, 0.25243562, -0.006...",0.881996,0.779528,[50],0.176213,0.472012,0.00001,2.0,0.01
9,Young,0.9,"[[-0.14466392, 0.019785453, -0.005838691, -0.2...",0.595986,0.738826,[50],0.620303,0.776660,0.00001,0.0,0.01


In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')